In [6]:
# ================================================
# Multi-frame 2×2 GIF Builder (Plain + Annotated)
# - UL: original_overlay (grayscale)           [label: Ground Truth]
# - UR: composite (ER=feature_0 → GREEN, Vesicles=feature_1 → MAGENTA)
# - LL: feature_0 (grayscale)                  [label: NanTex ER]
# - LR: feature_1 (grayscale)                  [label: NanTex Vesicles]
# - PLAIN GIF: weiße Trennlinien + 5 µm Scalebars (keine Labels, kein Timestamp)
# - ANNOTATED GIF: weiße Labels (ohne Hintergrund), Timestamp ZENTRIERT OBERHALB der Panels,
#                  außerdem weiße Trennlinien + 5 µm Scalebars
# - Pixelgröße = 42.7 nm/px, Abspielgeschwindigkeit = 260 ms/Frame
# - Fortschrittsbalken + Sanity-Outputs
# ================================================

# Falls nötig, installieren:
# %pip install pillow numpy tqdm

import os
import re
from typing import Tuple, Optional, List, Dict, Any
import numpy as np
from PIL import Image, ImageDraw, ImageFont
try:
    from tqdm.autonotebook import tqdm
except Exception:
    from tqdm import tqdm  # fallback


# =========================
# Configuration
# =========================
BASE_PATH = r"C:\Users\theni\Desktop\FSU\Nantex\ERonly\collection\collection"
OUT_GIF_PLAIN = "combined_all_2x2_plain.gif"
OUT_GIF_ANNOT = "combined_all_2x2_annotated.gif"

# Playback
FRAME_DURATION_MS = 260   # ms pro Frame (real-time)
LOOP_COUNT = 0            # 0 = Endlosschleife

# Pixelgröße & Scalebar
PIXEL_SIZE_NM = 42.7      # nm/px
SCALEBAR_UM = 5.0         # µm
SCALEBAR_THICK = 3        # px
SCALEBAR_MARGIN = 10      # px vom Panelrand
SCALEBAR_TEXT = "5 µm"

# Separator-Linien (beide GIFs)
SEPARATOR_COLOR = (255, 255, 255)
SEPARATOR_THICK = 1

# Panel-Labels (nur annotated; weißer Text, KEIN Hintergrund)
LABELS = {
    "UL": "Ground Truth",
    "UR": "NanTex Demixed",
    "LL": "NanTex ER",
    "LR": "NanTex Vesicles",
}
LABEL_TEXT_COLOR = (255, 255, 255)
LABEL_OFFSET = (8, 8)  # Offset relativ zur Panel-Ecke (oben links)

# Timestamp (nur annotated; weißer Text, ZENTRIERT OBERHALB der Panels, kein Hintergrund)
TIMESTAMP_TEXT_COLOR = (255, 255, 255)
TIMESTAMP_TOP_MARGIN = 6  # px vom oberen Rand des Mosaiks


# =========================
# Utilities
# =========================
REQUIRED_FILES = {
    "original": "original_overlay",
    "f0": "feature_0",      # ER
    "f1": "feature_1",      # Vesicles
}
OPTIONAL_FILES = {
    "dream": "dream_overlay",
}

def get_font(size: int) -> ImageFont.ImageFont:
    """Versuche TTF; Fallback zu Default."""
    try:
        return ImageFont.truetype("arial.ttf", size=size)
    except Exception:
        try:
            return ImageFont.truetype("DejaVuSans.ttf", size=size)
        except Exception:
            return ImageFont.load_default()

def find_png(path_without_ext: str) -> Optional[str]:
    """Suche stem.png / stem.PNG."""
    candidates = [f"{path_without_ext}.png", f"{path_without_ext}.PNG"]
    for p in candidates:
        if os.path.isfile(p):
            return p
    return None

def load_grayscale(path: str, size: Optional[Tuple[int, int]] = None) -> Image.Image:
    """Lade als 8-bit 'L', optional resize auf (W,H)."""
    img = Image.open(path)
    if size is not None:
        img = img.resize(size, Image.BILINEAR)
    img = img.convert("L")
    return img

def normalize_to_float(arr: np.ndarray) -> np.ndarray:
    """[0..255] → [0..1] mit robustem Handling flacher Bilder."""
    arr = arr.astype(np.float32)
    vmin = float(arr.min())
    vmax = float(arr.max())
    if vmax <= vmin + 1e-6:
        return np.zeros_like(arr, dtype=np.float32)
    return (arr - vmin) / (vmax - vmin)

def color_composite_feature_green_magenta_overlay(f0_L: Image.Image, f1_L: Image.Image) -> Image.Image:
    """
    Alpha-composited overlay:
      - ER (feature_0) = background GREEN
      - Vesicles (feature_1) = MAGENTA overlay in front
      - Vesicles are twice as strong as ER (w_ves = 2.0 vs w_er = 1.0)
    Implementation:
      base (ER) color    = [0, I_er, 0]
      overlay (ves) col  = [I_v, 0, I_v]
      alpha (ves)        = I_v     (opacity grows with vesicle intensity)
      composite          = (1 - alpha) * base + alpha * overlay
    Inputs:  8-bit grayscale 'L'
    Output:  8-bit 'RGB'
    """
    # to arrays
    f0 = np.array(f0_L, dtype=np.uint8)  # ER
    f1 = np.array(f1_L, dtype=np.uint8)  # Vesicles

    # normalize to 0..1
    f0n = normalize_to_float(f0)
    f1n = normalize_to_float(f1)

    # strength weights
    w_er  = 1.0
    w_ves = 2.0  # "twice as strong as ER"

    # intensities (clipped to 0..1)
    I_er = np.clip(f0n * w_er,  0.0, 1.0)
    I_v  = np.clip(f1n * w_ves, 0.0, 1.0)

    # base green (ER) and magenta overlay (vesicles)
    base   = np.stack([np.zeros_like(I_er), I_er, np.zeros_like(I_er)], axis=-1)  # [0, I_er, 0]
    over   = np.stack([I_v, np.zeros_like(I_v), I_v], axis=-1)                    # [I_v, 0, I_v]
    alpha  = I_v[..., None]  # use boosted vesicle intensity as opacity

    # alpha compositing: vesicles "in front"
    rgb_f = (1.0 - alpha) * base + alpha * over
    rgb   = np.clip(rgb_f * 255.0, 0, 255).astype(np.uint8)
    return Image.fromarray(rgb, mode="RGB")


def make_mosaic_2x2(ul: Image.Image, ur: Image.Image, ll: Image.Image, lr: Image.Image) -> Image.Image:
    """Mosaik 2×2: [ul|ur; ll|lr]."""
    ul_rgb = ul.convert("RGB")
    ur_rgb = ur.convert("RGB")
    ll_rgb = ll.convert("RGB")
    lr_rgb = lr.convert("RGB")
    w, h = ul_rgb.size
    canvas = Image.new("RGB", (2 * w, 2 * h))
    canvas.paste(ul_rgb, (0, 0))
    canvas.paste(ur_rgb, (w, 0))
    canvas.paste(ll_rgb, (0, h))
    canvas.paste(lr_rgb, (w, h))
    return canvas

def image_stats_u8(img_L: Image.Image) -> Dict[str, Any]:
    """Min/Max/Mean/Flat/Size für 8-bit 'L'."""
    arr = np.array(img_L, dtype=np.uint8)
    return {
        "min": int(arr.min()),
        "max": int(arr.max()),
        "mean": float(arr.mean()),
        "flat": bool(arr.max() == arr.min()),
        "size": img_L.size
    }

def overlap_percentage(f0_L: Image.Image, f1_L: Image.Image, threshold: int = 1) -> float:
    """% Pixel mit f0>thr und f1>thr (einfaches Overlap-Maß)."""
    a = np.array(f0_L, dtype=np.uint8)
    b = np.array(f1_L, dtype=np.uint8)
    both = np.logical_and(a > threshold, b > threshold).sum()
    total = a.size
    return 100.0 * both / float(total) if total else 0.0

def natural_sort_key(s: str):
    """Natürliche Sortierung."""
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r'(\d+)', s)]


# =========================
# Drawing helpers
# =========================
def draw_separator_lines(canvas: Image.Image, thick: int = 1, color=(255, 255, 255)) -> None:
    w, h = canvas.size
    mid_x = w // 2
    mid_y = h // 2
    draw = ImageDraw.Draw(canvas)
    # vertikal
    draw.rectangle([mid_x - thick // 2, 0, mid_x + (thick - 1) // 2, h - 1], fill=color)
    # horizontal
    draw.rectangle([0, mid_y - thick // 2, w - 1, mid_y + (thick - 1) // 2], fill=color)

def draw_label_text(canvas: Image.Image, text: str, top_left: Tuple[int, int], font: ImageFont.ImageFont) -> None:
    """Einfacher weißer Label-Text ohne Hintergrund."""
    draw = ImageDraw.Draw(canvas    )
    draw.text(top_left, text, fill=(255, 255, 255), font=font)

def draw_timestamp_centered(canvas: Image.Image, text: str, font: ImageFont.ImageFont) -> None:
    """Weißer Timestamp, mittig oben über den Panels (ohne Hintergrund)."""
    draw = ImageDraw.Draw(canvas)
    W, _ = canvas.size
    try:
        bbox = draw.textbbox((0, 0), text, font=font)
        tw = bbox[2] - bbox[0]
        th = bbox[3] - bbox[1]
    except Exception:
        tw, th = draw.textsize(text, font=font)
    x = (W - tw) // 2
    y = TIMESTAMP_TOP_MARGIN
    draw.text((x, y), text, fill=TIMESTAMP_TEXT_COLOR, font=font)

def draw_scalebar_in_panel(canvas: Image.Image, panel_origin: Tuple[int, int], panel_size: Tuple[int, int],
                           scalebar_px: int, thick: int, label: str, font: ImageFont.ImageFont) -> None:
    """Scalebar unten rechts INSIDE eines Panels + weiße Beschriftung darüber (ohne BG)."""
    draw = ImageDraw.Draw(canvas)
    x0, y0 = panel_origin
    pw, ph = panel_size

    bar_x2 = x0 + pw - SCALEBAR_MARGIN
    bar_x1 = bar_x2 - scalebar_px
    bar_y2 = y0 + ph - SCALEBAR_MARGIN
    bar_y1 = bar_y2 - thick + 1

    bar_x1 = max(bar_x1, x0 + SCALEBAR_MARGIN)
    if bar_x2 <= bar_x1:
        return

    draw.rectangle([bar_x1, bar_y1, bar_x2, bar_y2], fill=(255, 255, 255))

    # Text über dem Balken
    #try:
    #    bbox = draw.textbbox((0, 0), label, font=font)
    #    tw = bbox[2] - bbox[0]
    #    th = bbox[3] - bbox[1]
    #except Exception:
    #    tw, th = draw.textsize(label, font=font)
    #tx = bar_x1 + int(max(0, (scalebar_px - tw)) // 2)
    #ty = bar_y1 - th - 4
    #draw.text((tx, ty), label, fill=(255, 255, 255), font=font)


# =========================
# Main execution
# =========================
if not os.path.isdir(BASE_PATH):
    raise FileNotFoundError(f"Base path does not exist or is not a directory: {BASE_PATH}")

out_gif_plain_path = os.path.join(BASE_PATH, OUT_GIF_PLAIN)
out_gif_annot_path = os.path.join(BASE_PATH, OUT_GIF_ANNOT)

# Folders sammeln & sortieren
folders = []
for dirpath, dirnames, filenames in os.walk(BASE_PATH):
    folders.append(dirpath)
folders = sorted(folders, key=natural_sort_key)
total_folders = len(folders)
print(f"Found {total_folders} folder(s) under base path.")

# Scalebar-Länge in Pixel
scalebar_px = int(round((SCALEBAR_UM * 1000.0) / PIXEL_SIZE_NM))  # µm → nm → px
print(f"Scalebar: {SCALEBAR_UM} µm entspricht {scalebar_px} px bei {PIXEL_SIZE_NM} nm/px.")

frames_plain: List[Image.Image] = []
frames_annot: List[Image.Image] = []
processed_info: List[Dict[str, Any]] = []
skipped: List[Dict[str, Any]] = []
ref_size = None
size_mismatches = 0

# Fonts (werden nach erster gültiger Größe bestimmt)
font_small = None
font_medium = None

for idx, folder in enumerate(tqdm(folders, desc="Processing folders")):
    original_path = find_png(os.path.join(folder, REQUIRED_FILES["original"]))
    f0_path = find_png(os.path.join(folder, REQUIRED_FILES["f0"]))
    f1_path = find_png(os.path.join(folder, REQUIRED_FILES["f1"]))

    missing = []
    if not original_path: missing.append("original_overlay.png")
    if not f0_path: missing.append("feature_0.png")
    if not f1_path: missing.append("feature_1.png")
    if missing:
        skipped.append({ "folder": folder, "reason": f"Missing: {', '.join(missing)}" })
        continue

    try:
        # Laden & Größenangleich
        original_img = Image.open(original_path).convert("L")
        if ref_size is None:
            ref_size = original_img.size
        elif original_img.size != ref_size:
            original_img = original_img.resize(ref_size, Image.BILINEAR)
            size_mismatches += 1

        f0_img = load_grayscale(f0_path, size=ref_size)  # ER
        f1_img = load_grayscale(f1_path, size=ref_size)  # Vesicles

        # Sanity
        stats_orig = image_stats_u8(original_img)
        stats_f0 = image_stats_u8(f0_img)
        stats_f1 = image_stats_u8(f1_img)
        ovlp = overlap_percentage(f0_img, f1_img, threshold=1)

        # Panels
        ul = original_img
        ur = color_composite_feature_green_magenta_overlay(f0_img, f1_img)
        ll = f0_img
        lr = f1_img

        # Basismosaik
        mosaic_base = make_mosaic_2x2(ul, ur, ll, lr)
        W, H = mosaic_base.size
        panel_w = W // 2
        panel_h = H // 2

        # Fonts initialisieren
        if font_small is None or font_medium is None:
            font_small = get_font(max(10, panel_h // 24))
            font_medium = get_font(max(12, panel_h // 16))

        # --------- PLAIN: Linien + Scalebars, KEINE Labels, KEIN Timestamp ---------
        mosaic_plain = mosaic_base.copy()
        draw_separator_lines(mosaic_plain, thick=SEPARATOR_THICK, color=SEPARATOR_COLOR)
        draw_scalebar_in_panel(mosaic_plain, (0, 0), (panel_w, panel_h), scalebar_px, SCALEBAR_THICK, SCALEBAR_TEXT, font_small)                 # UL
        draw_scalebar_in_panel(mosaic_plain, (panel_w, 0), (panel_w, panel_h), scalebar_px, SCALEBAR_THICK, SCALEBAR_TEXT, font_small)          # UR
        draw_scalebar_in_panel(mosaic_plain, (0, panel_h), (panel_w, panel_h), scalebar_px, SCALEBAR_THICK, SCALEBAR_TEXT, font_small)          # LL
        draw_scalebar_in_panel(mosaic_plain, (panel_w, panel_h), (panel_w, panel_h), scalebar_px, SCALEBAR_THICK, SCALEBAR_TEXT, font_small)    # LR
        frames_plain.append(mosaic_plain)

        # --------- ANNOTATED: Labels (weiß), Timestamp zentriert oben, Linien + Scalebars ---------
        mosaic_annot = mosaic_base.copy()
        draw_separator_lines(mosaic_annot, thick=SEPARATOR_THICK, color=SEPARATOR_COLOR)

        # Labels
        draw_label_text(mosaic_annot, LABELS["UL"], (LABEL_OFFSET[0], LABEL_OFFSET[1]), font_medium)                                  # UL
        draw_label_text(mosaic_annot, LABELS["UR"], (panel_w + LABEL_OFFSET[0], LABEL_OFFSET[1]), font_medium)                         # UR
        draw_label_text(mosaic_annot, LABELS["LL"], (LABEL_OFFSET[0], panel_h + LABEL_OFFSET[1]), font_medium)                         # LL
        draw_label_text(mosaic_annot, LABELS["LR"], (panel_w + LABEL_OFFSET[0], panel_h + LABEL_OFFSET[1]), font_medium)               # LR

        # Scalebars
        draw_scalebar_in_panel(mosaic_annot, (0, 0), (panel_w, panel_h), scalebar_px, SCALEBAR_THICK, SCALEBAR_TEXT, font_small)                 # UL
        draw_scalebar_in_panel(mosaic_annot, (panel_w, 0), (panel_w, panel_h), scalebar_px, SCALEBAR_THICK, SCALEBAR_TEXT, font_small)          # UR
        draw_scalebar_in_panel(mosaic_annot, (0, panel_h), (panel_w, panel_h), scalebar_px, SCALEBAR_THICK, SCALEBAR_TEXT, font_small)          # LL
        draw_scalebar_in_panel(mosaic_annot, (panel_w, panel_h), (panel_w, panel_h), scalebar_px, SCALEBAR_THICK, SCALEBAR_TEXT, font_small)    # LR

        # Timestamp mittig oben
        #timestamp_text = f"Frame {len(frames_annot) + 1}"
        #draw_timestamp_centered(mosaic_annot, timestamp_text, font_medium)

        frames_annot.append(mosaic_annot)

        processed_info.append({
            "folder": folder,
            "original": stats_orig,
            "feature_0": stats_f0,
            "feature_1": stats_f1,
            "overlap_pct": ovlp
        })

    except Exception as e:
        skipped.append({ "folder": folder, "reason": f"Error: {e}" })
        continue

# Speichern
if not frames_plain:
    print("No qualifying subfolders were found. Nothing to save.")
else:
    first_p, rest_p = frames_plain[0], frames_plain[1:]
    first_p.save(
        out_gif_plain_path,
        format="GIF",
        save_all=True,
        append_images=rest_p,
        duration=FRAME_DURATION_MS,
        loop=LOOP_COUNT,
        optimize=True,
    )
    print(f"[DONE] Saved PLAIN animated GIF with {len(frames_plain)} frame(s) to: {out_gif_plain_path}")

    first_a, rest_a = frames_annot[0], frames_annot[1:]
    first_a.save(
        out_gif_annot_path,
        format="GIF",
        save_all=True,
        append_images=rest_a,
        duration=FRAME_DURATION_MS,
        loop=LOOP_COUNT,
        optimize=True,
    )
    print(f"[DONE] Saved ANNOTATED animated GIF with {len(frames_annot)} frame(s) to: {out_gif_annot_path}")

# Sanity Summary
print("\n=== SANITY SUMMARY ===")
print(f"Total folders scanned: {total_folders}")
print(f"Frames created: {len(frames_plain)}")
print(f"Folders skipped: {len(skipped)}")
if len(skipped) > 0:
    print("Examples of skipped folders/reasons (up to 10):")
    for rec in skipped[:10]:
        print(f" - {rec['folder']}: {rec['reason']}")
if ref_size is not None:
    print(f"Reference panel size (from first valid folder): {ref_size}")
print(f"Images resized due to size mismatch (count): {size_mismatches}")

show_n = min(5, len(processed_info))
if show_n > 0:
    print(f"\nPer-frame stats for first {show_n} frame(s):")
    for i in range(show_n):
        rec = processed_info[i]
        print(f"Frame {i+1}: {rec['folder']}")
        print(f"  original_overlay  -> min={rec['original']['min']} max={rec['original']['max']} mean={rec['original']['mean']:.2f}")
        print(f"  feature_0 (ER)    -> min={rec['feature_0']['min']} max={rec['feature_0']['max']} mean={rec['feature_0']['mean']:.2f}")
        print(f"  feature_1 (Vesc.) -> min={rec['feature_1']['min']} max={rec['feature_1']['max']} mean={rec['feature_1']['mean']:.2f}")
        print(f"  overlap (f0 & f1) -> {rec['overlap_pct']:.2f}% of pixels > threshold")


Found 101 folder(s) under base path.
Scalebar: 5.0 µm entspricht 117 px bei 42.7 nm/px.


Processing folders:   0%|          | 0/101 [00:00<?, ?it/s]

[DONE] Saved PLAIN animated GIF with 100 frame(s) to: C:\Users\theni\Desktop\FSU\Nantex\ERonly\collection\collection\combined_all_2x2_plain.gif
[DONE] Saved ANNOTATED animated GIF with 100 frame(s) to: C:\Users\theni\Desktop\FSU\Nantex\ERonly\collection\collection\combined_all_2x2_annotated.gif

=== SANITY SUMMARY ===
Total folders scanned: 101
Frames created: 100
Folders skipped: 1
Examples of skipped folders/reasons (up to 10):
 - C:\Users\theni\Desktop\FSU\Nantex\ERonly\collection\collection: Missing: original_overlay.png, feature_0.png, feature_1.png
Reference panel size (from first valid folder): (512, 512)
Images resized due to size mismatch (count): 0

Per-frame stats for first 5 frame(s):
Frame 1: C:\Users\theni\Desktop\FSU\Nantex\ERonly\collection\collection\dream_0
  original_overlay  -> min=0 max=255 mean=15.22
  feature_0 (ER)    -> min=0 max=255 mean=20.53
  feature_1 (Vesc.) -> min=0 max=255 mean=1.64
  overlap (f0 & f1) -> 2.14% of pixels > threshold
Frame 2: C:\Users\th